In [2]:
# IMPORTS
import xarray as xr 
import matplotlib.pyplot as plt
import gsw
import pandas as pd
import numpy as np
import os
import copernicus_marine_client as copernicus_marine

In [3]:
def compute_AS(row):
    return gsw.SA_from_SP(row['PSAL'], row['PRES'], row['LONGITUDE'], row['LATITUDE'])


def compute_CT(row):
    """
    Computes Conservative temperature from In-situ temperature, absolute salinity and pressure
    """
    return gsw.CT_from_t(row['ASAL'], row['TEMP'], row['PRES'])


def compute_pres(row):
    return gsw.p_from_z(row['HEIGHT'], row['LATITUDE'], 0, 0)

In [4]:
# ARGO POINTS
year = '2020'

argo_dir = 'argos_collocated'
outdir = 'model_surface'
model_dir = './tmp'

def extract_model_profile(argo_dir, outdir, model_dir, file):
    date = file[0:8]
    if(not os.path.isfile(outdir+"/"+date+'.feather')):
        options_df = pd.read_feather(argo_dir+'/'+file)
        dims = ['ILAT_MODEL', 'ILON_MODEL', 'IHEIGHT_MODEL']  # indexes in the model dataset

        options_df = options_df.drop_duplicates(subset=dims)

        points = options_df[dims] 

        points['ILON_MODEL'] = points['ILON_MODEL']%1440

        # Read models and join them as they have the same coordinates
        model = xr.open_dataset(model_dir+
                "/cmems_mod_glo_phy-all_my_0.25deg_P1D-m-"+date+".nc")

        psal = model['so_oras'].values[0, points['IHEIGHT_MODEL'], points['ILAT_MODEL'], points['ILON_MODEL']]
        temp = model['thetao_oras'].values[0, points['IHEIGHT_MODEL'],
                                        points['ILAT_MODEL'], points['ILON_MODEL']]

        arrays = [
                options_df['LATITUDE'],
                options_df['LONGITUDE'],
                options_df['HEIGHT'],
                options_df['HEIGHT_MODEL'],
                psal,
                temp]

        df = pd.DataFrame(
                {
                    row_index: row_array
                    for row_index, row_array in enumerate(arrays)
                }
            )

        del options_df
        del psal
        del temp
        del arrays
        del model

        df.dropna(inplace=True)
        df.rename(columns={0: "LATITUDE", 1: "LONGITUDE", 2: "HEIGHT", 3:"HEIGHT_MODEL", 4: "PSAL", 5: "TEMP"}, inplace=True)

        df['PRES'] = df.apply(compute_pres, axis=1)
        df['ASAL'] = df.apply(compute_AS, axis=1)
        df['CTEMP'] = df.apply(compute_CT, axis=1)
        df = df.drop(['PSAL', 'TEMP'], axis=1).reset_index()
        df.to_feather(outdir+"/"+date+'.feather')


def extract_model_surface(outdir, model_dir, file):
    date = file[0:8]
    if(not os.path.isfile(outdir+"/"+date+'.feather')):
        # Read models and join them as they have the same coordinates
        model = xr.open_dataset(model_dir+
                "/cmems_mod_glo_phy-all_my_0.25deg_P1D-m-"+date+".nc")

        sss = model['so_oras'][0, 0, :, :]
        sst = model['thetao_oras'][0, 0, :, :]
        vo = model['vo_oras'][0, 0, :, :]
        uo = model['uo_oras'][0, 0, :, :]
        zo = model['zos_oras'][0, :, :]
        mlt = model['mlotst_oras'][0, :, :]

        sss.to_netcdf(outdir+"/"+date+'_sss.nc')
        sst.to_netcdf(outdir+"/"+date+'_sst.nc')
        vo.to_netcdf(outdir+"/"+date+'_vo.nc')
        uo.to_netcdf(outdir+"/"+date+'_uo.nc')
        zo.to_netcdf(outdir+"/"+date+'_ssh.nc')
        mlt.to_netcdf(outdir+"/"+date+'_mlt.nc')

for file in os.listdir(argo_dir):
    if year in file:
        try:
            date = file[0:8]
            year2 = file[0:4]
            print(date)
            if not os.path.isfile(outdir+'/'+year2+'/'+date+'.feather'):
                # Load xarray dataset
                sst_l3s = copernicus_marine.get(
                    dataset_id = "cmems_mod_glo_phy-all_my_0.25deg_P1D-m",
                    username = "",
                    password = "",
                    force_download = True,
                    filter = '*'+date+'*.nc*',
                    output_directory= model_dir,
                    no_directories = True
                )
                extract_model_profile(argo_dir, outdir, model_dir, file)
                extract_model_surface(outdir, model_dir, file)
                os.remove(model_dir+"/cmems_mod_glo_phy-all_my_0.25deg_P1D-m-"+date+".nc")
        except Exception as error:
            print("err: " , error)

20200511
ERROR - 2024-04-11T08:07:02Z - Client version 0.10.1 is not compatible with current backend service. Please update to the latest client version.
ERROR - 2024-04-11T08:07:02Z - Client version 0.10.1 is not compatible with current backend service. Please update to the latest client version.
ERROR - 2024-04-11T08:07:02Z - Client version 0.10.1 is not compatible with current backend service. Please update to the latest client version.
INFO - 2024-04-11T08:07:04Z - Selected dataset version is "202311"
INFO - 2024-04-11T08:07:04Z - Downloading using service original-files...


100%|██████████| 1/1 [00:13<00:00, 13.58s/it]
